# Dependencies

3d party packages:
* cvat-sdk - CVAT Python API
* python-dotenv - environment variables from .env file (just like docker-compose)

In [ ]:
%%capture
%pip install -U cvat-sdk python-dotenv

# Paths

In [ ]:
from os import makedirs, path as osp


PATH_DATA = osp.join('.', 'data')
PATH_ANNOTATIONS = osp.join(PATH_DATA, 'annotations')
PATH_IMAGES = osp.join(PATH_DATA, 'images')
PATH_LABELS = osp.join(PATH_DATA, 'labels')

makedirs(PATH_ANNOTATIONS, exist_ok=True)
makedirs(PATH_IMAGES, exist_ok=True)
makedirs(PATH_LABELS, exist_ok=True)

In [ ]:
%ls {PATH_ANNOTATIONS} {PATH_IMAGES} {PATH_LABELS}

# Annotations

Annotations are being processed with CVAT. Use CVAT SDK (Python API) to get annotations.

In [ ]:
# FIXME: remove (debug only)
# client = Client(CVAT_SERVER, config=Config(verify_ssl=False))
# client.login((CVAT_USER, CVAT_PASSWORD))

In [ ]:
# ...
# client.close()

## Annotations retrieval

CVAT SDK was chosen over Datumaro (and static annotations from GitHub) because of ability to choose annotation based on job status (annotations only from 'completed' jobs are required).

In [ ]:
from json import loads
from urllib3.exceptions import InsecureRequestWarning
from warnings import catch_warnings, filterwarnings

from cvat_sdk import Client, Config


def get_annotations(host, user, password):
    with catch_warnings():
        # Suppress insecure SSL warnings
        filterwarnings("ignore", category=InsecureRequestWarning)

        # Create a config instance
        config = Config(verify_ssl=False)

        # Create a client instance
        client = Client(host, config=config)

        # Log in to the CVAT server
        client.login((user, password))

        projects = loads(client.projects.list(return_json=True))
        projects = {
            project['id']: project
            for project in sorted(projects, key=lambda p: p['id'])
        }
        jobs = loads(client.jobs.list(return_json=True))
        jobs_completed = {
            job['id']: job for job in jobs if job['status'] == 'completed'
        }

        # Download the annotations for each completed task
        annotations = []
        labels_project = {}
        for job_id in jobs_completed:
            job = client.jobs.retrieve(int(job_id))
            meta = job.get_meta().to_dict()  # frame index info
            labels = {
                label['id']: label
                for label in map(lambda l: l.to_dict(), job.get_labels())
            }
            jobs_completed[job_id]['labels']['extra'] = labels
            annotation = job.get_annotations().to_dict()['shapes']
            # Hope frames are returned from the server in correct order
            frames = job.get_frames_info()
            [a.update({
                'project_id': jobs_completed[job_id]['project_id'],  # project id to map labels
                'project_name': projects[jobs_completed[job_id]['project_id']]['name'],
                'job_id': job_id,  # job id to rule them all
                'label': labels[a['label_id']]['name'],  # label name
                'color': labels[a['label_id']]['color']  # and original color
            }) for a in annotation]
            [a.update(
                frames[a['frame'] - meta['start_frame']].to_dict()
            ) for a in annotation]
            annotations.append(annotation)

        # Close connection
        client.logout()

    return sum(annotations, []), jobs_completed

## Fetch annotations from CVAT

Set up environment variables for `CVAT_SERVER`, `CVAT_USER`, `CVAT_PASSWORD` in bash shell, .env file or in the cell below.

In [ ]:
from getpass import getpass
from os import environ

from dotenv import load_dotenv


load_dotenv()  # take environment variables from .env (not overwrite existing)

# CVAT host address
CVAT_SERVER = environ.get('CVAT_SERVER', None) or ''
assert CVAT_SERVER, f"Provide CVAT_SERVER environment variable or set above!"

# CVAT user to login
CVAT_USER = environ.get('CVAT_USER', None) or ''
assert CVAT_USER, f"Provide CVAT_USER environment variable or set above!"

# CVAT password to login with
CVAT_PASSWORD = (
    environ.get('CVAT_PASSWORD', None) or
    getpass(f"Your password for {CVAT_USER}@{CVAT_SERVER}:")
)
if not CVAT_PASSWORD:
    print(f"WARNING: login to CVAT server with empty password was not tested!")

Fetch annotations from CVAT.

In [ ]:
annotations, jobs_completed = get_annotations(
    CVAT_SERVER, CVAT_USER, CVAT_PASSWORD
)

## Extract labels per project

In [ ]:
project_labels = {}
for job_id, job in jobs_completed.items():
    project_labels[job['project_id']] = sorted(job['labels']['extra'].values(), key=lambda v: v['id'])
project_labels

## Create YOLO annotations

Generate YOLO annotations via DataFrames.

In [ ]:
import pandas as pd


frame_annotations = pd.DataFrame(annotations)

frame_annotations

In [ ]:
# frame_annotations.columns

### Get label id to name mapping

In [ ]:
frame_annotations[['label_id', 'project_name']].drop_duplicates()

### DataFrame with label scheme

Labels in CVAT are enumerated across all projects, so one needs to enumerate per-project.

In [ ]:
frame_labels = pd.DataFrame(sum(project_labels.values(), []))

frame_labels = pd.merge(
    frame_labels, frame_annotations[['project_id', 'project_name']].drop_duplicates(),
    on='project_id', how='left'
)

frame_labels.sort_values('id')

In [ ]:
# frame_labels.columns

### Make CVAT to YOLO label mapping

In [ ]:
mapping_labels = {}

groups = frame_labels.groupby('project_id')

for project_id, group in groups:
    yolo_ids = dict(enumerate(group.sort_values('id')['id']))
    mapping_labels.update({v: k for k, v in yolo_ids.items()})

frame_labels['yolo_id'] = frame_labels['id'].apply(lambda x: mapping_labels[x])

frame_labels

In [ ]:
PATH_YOLO = osp.join(PATH_DATA, 'yolo')
makedirs(PATH_YOLO, exist_ok=True)

groups = frame_annotations.groupby('name')
projects = {}  # meta info

# Create YOLO annotations per image and fill meta info
for filename, group in groups:
    filename_source = osp.join(PATH_IMAGES, filename)
    if not osp.isfile(filename_source):
        continue
    project_name = group['project_name'].unique().item()
    project_dict = projects.get(project_name, {})
    project_tasks = project_dict.get('tasks', {})
    project_groups = project_dict.get('groups', [])
    project_labels = project_dict.get('labels', [])
    path_base = PATH_LABELS  # osp.join(PATH_LABELS, project_name)
    filename_target = osp.join(path_base, filename)
    # Project (CVAT project) -> images (CVAT task) -> item (CVAT image)
    project_task = osp.dirname(filename)
    project_task_images = project_tasks.get(project_task, [])
    project_task_images.append(osp.join('.', osp.basename(PATH_IMAGES), filename))
    filename_target = f"{osp.splitext(filename_target)[0]}.txt"
    # filename_labels = f"{path_base}.txt"
    makedirs(osp.dirname(filename_target), exist_ok=True)
    with open(filename_target, 'w') as target:
        items = []
        for index, row in group.iterrows():
            if row['type'] not in ['rectangle']:
                continue  # TODO: render polygons to bboxes
            x1, y1, x2, y2 = row['points']
            w, h = row['width'], row['height']
            items.append(
                f"{mapping_labels[row['label_id']]}\t"
                f"{(x1 + x2) / 2 / w:.5f} "
                f"{(y1 + y2) / 2 / h:.5f} "
                f"{(x2 - x1) / w:.5f} "
                f"{(y2 - y1) / h:.5f}"
            )
        target.write('\n'.join(items))
    if not project_labels:
        project_labels.extend(frame_labels[
            frame_labels['project_name'] == project_name
        ].sort_values('yolo_id')['name'].tolist())
    group['task_name'] = project_task
    project_groups.append(group)
    project_tasks[project_task] = project_task_images
    project_dict['tasks'] = project_tasks
    project_dict['groups'] = project_groups
    project_dict['labels'] = project_labels
    projects[project_name] = project_dict
    # with open(filename_labels, 'w') as labels:
    #     labels.write('\n'.join(
    #         frame_labels[
    #             frame_labels['project_name'] == project_name
    #         ].sort_values('yolo_id')['name'].tolist()
    #     ))

# Create image and label lists from meta info
for name, project in projects.items():
    for task, images in project['tasks'].items():
        with open(osp.join(PATH_DATA, f"{task}.txt"), 'w') as target:
            target.write('\n'.join(images))
    with open(osp.join(PATH_YOLO, f"{name}-labels.txt"), 'w') as target:
        target.write('\n'.join(project['labels']))
    project['groups'] = pd.concat(project['groups'])

> TODO: COCO annotations in PATH_ANNOTATIONS per project.

In [ ]:
from pprint import pprint

pprint(projects)

## Split training and validation

> TODO: K-fold cross-validation configs for each fold (K-fold training).

In [ ]:
from yaml import dump

from sklearn.model_selection import StratifiedGroupKFold


PATH_CONFIG = osp.join(PATH_DATA, 'config')
makedirs(PATH_CONFIG, exist_ok=True)

sgkf5 = StratifiedGroupKFold(n_splits=5)

for name, project in projects.items():
    frame_groups = project['groups']
    frame_groups = frame_groups[frame_groups['type'] == 'rectangle']  # TODO: polygons
    index_train, index_valid = next(iter(
        sgkf5.split(frame_groups, frame_groups['label_id'],
                    groups=frame_groups['task_name'])
    ))
    items_train = frame_groups.iloc[index_train]['task_name'].unique().tolist()
    items_train = [
        f"{osp.join('.', osp.basename(PATH_DATA), item)}.txt"
        for item in items_train
    ]
    items_valid = frame_groups.iloc[index_valid]['task_name'].unique().tolist()
    items_valid = [
        f"{osp.join('.', osp.basename(PATH_DATA), item)}.txt"
        for item in items_valid
    ]
    config = {
        'train': items_train,
        'val': items_valid,
        'test': items_valid,
        'nc': len(project['labels']),
        'names': project['labels']
    }
    with open(osp.join(PATH_CONFIG, f"{name}.yaml"), 'w') as target:
        dump(config, target)

In [ ]:
%ls {PATH_ANNOTATIONS}/*

In [ ]:
%ls {PATH_LABELS}/*